# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - API Development

**Subtopic** - Building a RESTful API with Flask

**Use Case** - Develop a Python module that simulates the behavior of a RESTful API using Flask. The module should include endpoints for creating, reading, updating, and deleting (CRUD) JSON data. The data should be stored in memory and not require any external databases.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt
Problem Statement:
- A small startup is building a prototype for a new web application, and they need a simple backend API to manage user-submitted data.
- Since the project is still in its early stages, they want to avoid the overhead of setting up a full database.
- Your task is to develop a Python module that simulates the behavior of a RESTful API using Flask.
- This API should support standard CRUD (Create, Read, Update, Delete) operations on JSON data, which will be stored entirely in memory.
- The goal is to provide a fully functional, lightweight REST API that can be used during development and testing, without relying on any external databases or persistent storage.

Function Signature:
```python
def create_app() -> Flask:
```

##Input Format:
The Flask API must support the following HTTP endpoints:
```python
1. POST /items
   - Description: Create a new item.
   - Request Body (JSON):
	{
	   "id": <integer>,         # unique identifier, positive integer
	   "name": <string>,        # non-empty string, max 100 characters
       "value": <float>         # positive float
    }

2. GET /items
   - Description: Retrieve all items.
   - Response: JSON list of all stored items.

3. GET /items/<id>
   - Description: Retrieve a single item by ID.
   - Response: JSON object representing the item.

4. PUT /items/<id>
   - Description: Update an existing item by ID.
   - Request Body (JSON):
     {
		"name": <string>,        # non-empty string, max 100 characters (optional for update)
		"value": <float>         # positive float (optional for update)
     }

5. DELETE /items/<id>
   - Description: Delete an item by ID.
```

##Input Constraints:
- All requests must be in the form of standard HTTP methods: GET, POST, PUT, DELETE.
- Item data must be valid JSON with the following structure:
    - "id": integer (unique and positive)
    - "name": non-empty string (max 100 characters)
    - "value": positive float
- POST and PUT requests must include a JSON body.
- GET and DELETE requests must not include a body.
- PUT and DELETE requests must refer to an existing item ID.
- No external databases should be used; all data must be stored in memory.

##Output Format:
The Flask app exposes the following RESTful JSON endpoints:
```python
1. POST /items/
   - Description: Create a new item.
   - Request Body (application/json): {"id": int, "name": str, "value": float}
   - Response: JSON of created item with HTTP 201 status.

2. GET /items
   - Description: Retrieve a list of all items.
   - Response: JSON list of all items with HTTP 200 status.

3. GET /items/<id>
   - Description: Retrieve a specific item by ID.
   - Response: JSON of the item with HTTP 200 status, or HTTP 404 if not found.

4. PUT /items/<id>
   - Description: Update an existing item.
   - Request Body (application/json): {"name": str, "value": float}
   - Response: JSON of updated item with HTTP 200 status, or HTTP 404 if not found.

5. DELETE /items/<id>
   - Description: Delete an item by ID.
   - Response: JSON with {"message": "Item deleted"} and HTTP 200 status,
     or HTTP 404 if not found.
```

##Error Response Format:
- Error Responses (400, 404)
 - Description: Returned for invalid requests or missing resources.
 - Response: JSON object with a 'message' field explaining the error.
 - Example: {"message": "Item with ID 10 not found."}.

##Example:
```python
Input:
POST /items
Body: {"id": 1, "name": "widget", "value": 99.99}

GET /items

GET /items/1

PUT /items/1
Body: {"name": "super-widget", "value": 149.99}

DELETE /items/1

Output:
POST /items -> 201 Created
{"id": 1, "name": "widget", "value": 99.99}

GET /items -> 200 OK
[{"id": 1, "name": "widget", "value": 99.99}]

GET /items/1 -> 200 OK
{"id": 1, "name": "widget", "value": 99.99}

PUT /items/1 -> 200 OK
{"id": 1, "name": "super-widget", "value": 149.99}

DELETE /items/1 -> 200 OK
{"message": "Item deleted"}
```


# Requirements
Explicit Requirements:
- The system must expose RESTful API endpoints using Flask.
- Supported operations: Create (POST), Read (GET), Update (PUT), Delete (DELETE).
- Data must be stored in memory only (no external databases).
- Each item must be represented as JSON with "id", "name", and "value" fields.
- The API must handle appropriate status codes:
    - 200 OK for successful GET/PUT/DELETE
    - 201 Created for successful POST
    - 400 Bad Request for invalid input
    - 404 Not Found for missing items
- IDs must be unique and positive integers.
- Names must be non-empty strings up to 100 characters.
- Values must be positive floats.

Implicit Requirements:
- Data storage should persist only for the lifetime of the Flask app (in-memory).
- Concurrency issues are not addressed and assumed to be out of scope.
- The server should return error messages in JSON format for invalid input.
- The API must be self-contained and testable without external dependencies.

Solution Expectations:
- Implement a single Python module using Flask to simulate the API.
- Use a global in-memory dictionary or list to store JSON data.
- Define endpoints under the route `/items` with the following behaviors:
    - `POST /items`: Create a new item with JSON body containing "id", "name", "value".
    - `GET /items`: Return a list of all items in JSON format.
    - `GET /items/<id>`: Return the item with the given ID, or 404 if not found.
    - `PUT /items/<id>`: Update an existing item with new "name" and/or "value".
    - `DELETE /items/<id>`: Delete the item with the given ID.
- Validate request payloads and return 400 errors for missing/invalid fields.
- Handle error cases gracefully with appropriate HTTP status and JSON messages.
- No external storage, database, or authentication should be used.

Edge Cases and Behavior:
- Creating an item with an existing ID should return 400 with an error message.
- Creating an item without required fields ("id", "name", "value") should return 400.
- Accessing a non-existent item via GET, PUT, or DELETE should return 404.
- Updating an item with no valid fields ("name" or "value") should return 400.
- Deleting an already deleted or unknown item should return 404.
- The system must preserve existing data across requests during server runtime.
- All responses must be in valid JSON format with correct content-type headers.

In [ ]:
# code
"""Flask app that implements an in-memory CRUD REST API for items."""

from flask import Flask, request, jsonify

items = {}


def create_app() -> Flask:
    """Create and return the Flask application."""
    app = Flask(__name__)

    def validate_item_data(data, is_post=True):
        """
        Validate the structure and content of item data.

        The 'is_post' flag determines if the validation is for a
        creation (POST) or update (PUT) request.
        """
        if not isinstance(data, dict):
            return False, "Request body must be a JSON object."

        if is_post:
            if (
                "id" not in data
                or "name" not in data
                or "value" not in data
            ):
                return (
                    False,
                    "Missing required fields: 'id', 'name', 'value'."
                )

            if not isinstance(data["id"], int):
                return False, "ID must be an integer."
            if data["id"] <= 0:
                return False, "ID must be a positive integer."
            if data["id"] in items:
                return (
                    False,
                    f"Item with ID {data['id']} already exists."
                )
            if (
                not isinstance(data["name"], str)
                or not data["name"]
                or len(data["name"]) > 100
            ):
                return (
                    False,
                    "Name must be a non-empty string and max 100 characters."
                )
            if (
                not isinstance(data["value"], float)
                or data["value"] <= 0
            ):
                return False, "Value must be a positive float."
        else:
            if not any(field in data for field in ["name", "value"]):
                return (
                    False,
                    "At least 'name' or 'value' must be provided for update."
                )

            if "name" in data:
                if (
                    not isinstance(data["name"], str)
                    or not data["name"]
                    or len(data["name"]) > 100
                ):
                    return (
                        False,
                        "Name must be a non-empty string and "
                        "max 100 characters."
                    )
            if "value" in data:
                if (
                    not isinstance(data["value"], float)
                    or data["value"] <= 0
                ):
                    return False, "Value must be a positive float."

        return True, None

    @app.route("/items", methods=["POST"])
    def create_item():
        """Create a new item."""
        data = request.get_json()
        if data is None:
            return jsonify({"message": "Request body must be JSON."}), 400

        is_valid, error_message = validate_item_data(data, is_post=True)
        if not is_valid:
            return jsonify({"message": error_message}), 400

        item = {
            "id": data["id"],
            "name": data["name"],
            "value": float(data["value"])
        }
        items[item["id"]] = item
        return jsonify(item), 201

    @app.route("/items", methods=["GET"])
    def get_all_items():
        """Return a list of all items."""
        return jsonify(list(items.values())), 200

    @app.route("/items/<int:item_id>", methods=["GET"])
    def get_item(item_id):
        """Return a single item by ID."""
        item = items.get(item_id)
        if item is None:
            return (
                jsonify({"message": f"Item with ID {item_id} not found."}),
                404
            )
        return jsonify(item), 200

    @app.route("/items/<int:item_id>", methods=["PUT"])
    def update_item(item_id):
        """Update an existing item by ID."""
        item = items.get(item_id)
        if item is None:
            return (
                jsonify({"message": f"Item with ID {item_id} not found."}),
                404
            )

        data = request.get_json()
        if data is None:
            return jsonify({"message": "Request body must be JSON."}), 400

        is_valid, error_message = validate_item_data(data, is_post=False)
        if not is_valid:
            return jsonify({"message": error_message}), 400

        if "name" in data:
            item["name"] = data["name"]
        if "value" in data:
            item["value"] = float(data["value"])

        items[item_id] = item
        return jsonify(item), 200

    @app.route("/items/<int:item_id>", methods=["DELETE"])
    def delete_item(item_id):
        """Delete an item by ID."""
        if item_id not in items:
            return (
                jsonify({"message": f"Item with ID {item_id} not found."}),
                404
            )

        del items[item_id]
        return jsonify({"message": "Item deleted"}), 200

    return app


if __name__ == "__main__":
    app = create_app()
    app.run(debug=True)


In [ ]:
# tests

import unittest
import json
from main import create_app

class TestFlaskRestAPI(unittest.TestCase):

    def setUp(self):
        try:
            self.app = create_app()
            self.app.config['TESTING'] = True
            self.client = self.app.test_client()

            self._clear_items_storage()

        except ImportError:
            self.app = None
            self.client = None

    def _find_working_endpoint(self):
        test_endpoints = ['/items', '/items/', '/api/items', '/api/items/']

        for endpoint in test_endpoints:
            try:
                response = self.client.get(endpoint)
                if response.status_code != 404:
                    return endpoint
            except:
                continue

        return '/items'

    def _create_item(self, item_data, endpoint=None):
        """Helper method to create an item, trying different endpoint variations."""
        if endpoint is None:
            endpoint = self._find_working_endpoint()

        response = self.client.post(endpoint,
                                  data=json.dumps(item_data),
                                  content_type='application/json')

        if response.status_code == 404:
            endpoints_to_try = ['/items', '/items/', '/api/items', '/api/items/']
            for ep in endpoints_to_try:
                response = self.client.post(ep,
                                          data=json.dumps(item_data),
                                          content_type='application/json')
                if response.status_code != 404:
                    break

        return response

    def _clear_items_storage(self):
        """Helper method to clear in-memory storage between tests."""
        if not self.app:
            return

        try:
            endpoint = self._find_working_endpoint()
            response = self.client.get(endpoint)
            if response.status_code == 200:
                items = json.loads(response.data)
                for item in items:
                    self.client.delete(f'{endpoint.rstrip("/")}/{item["id"]}')
        except:
            pass


    def test_create_item_success(self):
        """Test successful creation of a new item via POST /items."""
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        response = self._create_item(item_data)

        if response.status_code != 201:
            print(f"DEBUG: Expected 201, got {response.status_code}")
            print(f"DEBUG: Response data: {response.data}")
            print(f"DEBUG: Response headers: {response.headers}")

            try:
                print("DEBUG: Available routes:")
                for rule in self.app.url_map.iter_rules():
                    print(f"  {rule.rule} -> {rule.methods}")
            except:
                print("DEBUG: Could not list available routes")

            try:
                error_data = json.loads(response.data)
                print(f"DEBUG: Error message: {error_data.get('message', 'No message')}")
            except:
                print("DEBUG: Could not parse response as JSON")

        self.assertEqual(response.status_code, 201)
        response_data = json.loads(response.data)
        self.assertEqual(response_data['id'], 1)
        self.assertEqual(response_data['name'], "widget")
        self.assertEqual(response_data['value'], 99.99)

    def test_create_item_duplicate_id(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        endpoint = self._find_working_endpoint()
        self.client.post(endpoint,
                        data=json.dumps(item_data),
                        content_type='application/json')

        response = self.client.post(endpoint,
                                  data=json.dumps(item_data),
                                  content_type='application/json')

        self.assertEqual(response.status_code, 400)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_create_item_missing_fields(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "name": "widget",
            "value": 99.99
        }

        endpoint = self._find_working_endpoint()
        response = self.client.post(endpoint,
                                  data=json.dumps(item_data),
                                  content_type='application/json')

        self.assertEqual(response.status_code, 400)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_create_item_invalid_id(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": -1,
            "name": "widget",
            "value": 99.99
        }

        endpoint = self._find_working_endpoint()
        response = self.client.post(endpoint,
                                  data=json.dumps(item_data),
                                  content_type='application/json')

        self.assertEqual(response.status_code, 400)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_create_item_invalid_name(self):
        if not self.client:
            self.skipTest("Flask app not available")

        endpoint = self._find_working_endpoint()

        item_data = {
            "id": 1,
            "name": "",
            "value": 99.99
        }

        response = self.client.post(endpoint,
                                  data=json.dumps(item_data),
                                  content_type='application/json')

        self.assertEqual(response.status_code, 400)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

        item_data = {
            "id": 2,
            "name": "a" * 101,
            "value": 99.99
        }

        response = self.client.post(endpoint,
                                  data=json.dumps(item_data),
                                  content_type='application/json')

        self.assertEqual(response.status_code, 400)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_create_item_invalid_value(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": -10.5
        }

        endpoint = self._find_working_endpoint()
        response = self.client.post(endpoint,
                                  data=json.dumps(item_data),
                                  content_type='application/json')

        self.assertEqual(response.status_code, 400)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_get_all_items_empty(self):
        if not self.client:
            self.skipTest("Flask app not available")

        endpoint = self._find_working_endpoint()
        response = self.client.get(endpoint)

        self.assertEqual(response.status_code, 200)
        response_data = json.loads(response.data)
        self.assertEqual(response_data, [])

    def test_get_all_items_with_data(self):
        if not self.client:
            self.skipTest("Flask app not available")

        items = [
            {"id": 1, "name": "widget1", "value": 99.99},
            {"id": 2, "name": "widget2", "value": 149.99}
        ]

        endpoint = self._find_working_endpoint()
        for item in items:
            self.client.post(endpoint,
                           data=json.dumps(item),
                           content_type='application/json')

        response = self.client.get(endpoint)

        self.assertEqual(response.status_code, 200)
        response_data = json.loads(response.data)
        self.assertEqual(len(response_data), 2)
        self.assertIn(items[0], response_data)
        self.assertIn(items[1], response_data)

    def test_get_single_item_success(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        create_response = self._create_item(item_data)
        self.assertEqual(create_response.status_code, 201)

        endpoint = self._find_working_endpoint()
        single_item_url = f"{endpoint.rstrip('/')}/1"

        response = self.client.get(single_item_url)

        self.assertEqual(response.status_code, 200)
        response_data = json.loads(response.data)
        self.assertEqual(response_data['id'], 1)
        self.assertEqual(response_data['name'], "widget")
        self.assertEqual(response_data['value'], 99.99)

    def test_get_single_item_not_found(self):
        if not self.client:
            self.skipTest("Flask app not available")

        endpoint = self._find_working_endpoint()
        single_item_url = f"{endpoint.rstrip('/')}/999"
        response = self.client.get(single_item_url)

        self.assertEqual(response.status_code, 404)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_update_item_success(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        endpoint = self._find_working_endpoint()
        self.client.post(endpoint,
                        data=json.dumps(item_data),
                        content_type='application/json')
        update_data = {
            "name": "super-widget",
            "value": 149.99
        }

        single_item_url = f"{endpoint.rstrip('/')}/1"
        response = self.client.put(single_item_url,
                                 data=json.dumps(update_data),
                                 content_type='application/json')

        self.assertEqual(response.status_code, 200)
        response_data = json.loads(response.data)
        self.assertEqual(response_data['id'], 1)
        self.assertEqual(response_data['name'], "super-widget")
        self.assertEqual(response_data['value'], 149.99)

    def test_update_item_partial_update(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        endpoint = self._find_working_endpoint()
        self.client.post(endpoint,
                        data=json.dumps(item_data),
                        content_type='application/json')


        update_data = {
            "name": "super-widget"
        }

        single_item_url = f"{endpoint.rstrip('/')}/1"
        response = self.client.put(single_item_url,
                                 data=json.dumps(update_data),
                                 content_type='application/json')

        self.assertEqual(response.status_code, 200)
        response_data = json.loads(response.data)
        self.assertEqual(response_data['name'], "super-widget")
        self.assertEqual(response_data['value'], 99.99)

    def test_update_item_not_found(self):
        if not self.client:
            self.skipTest("Flask app not available")

        update_data = {
            "name": "new-widget",
            "value": 199.99
        }

        endpoint = self._find_working_endpoint()
        single_item_url = f"{endpoint.rstrip('/')}/999"
        response = self.client.put(single_item_url,
                                 data=json.dumps(update_data),
                                 content_type='application/json')

        self.assertEqual(response.status_code, 404)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_update_item_no_valid_fields(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        endpoint = self._find_working_endpoint()
        self.client.post(endpoint,
                        data=json.dumps(item_data),
                        content_type='application/json')

        update_data = {
            "invalid_field": "test"
        }

        single_item_url = f"{endpoint.rstrip('/')}/1"
        response = self.client.put(single_item_url,
                                 data=json.dumps(update_data),
                                 content_type='application/json')

        self.assertEqual(response.status_code, 400)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_delete_item_success(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        endpoint = self._find_working_endpoint()
        self.client.post(endpoint,
                        data=json.dumps(item_data),
                        content_type='application/json')

        single_item_url = f"{endpoint.rstrip('/')}/1"
        response = self.client.delete(single_item_url)

        self.assertEqual(response.status_code, 200)
        response_data = json.loads(response.data)
        self.assertEqual(response_data['message'], "Item deleted")

        get_response = self.client.get(single_item_url)
        self.assertEqual(get_response.status_code, 404)

    def test_delete_item_not_found(self):
        if not self.client:
            self.skipTest("Flask app not available")

        endpoint = self._find_working_endpoint()
        single_item_url = f"{endpoint.rstrip('/')}/999"
        response = self.client.delete(single_item_url)

        self.assertEqual(response.status_code, 404)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_delete_already_deleted_item(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        endpoint = self._find_working_endpoint()
        self.client.post(endpoint,
                        data=json.dumps(item_data),
                        content_type='application/json')

        single_item_url = f"{endpoint.rstrip('/')}/1"
        self.client.delete(single_item_url)

        response = self.client.delete(single_item_url)

        self.assertEqual(response.status_code, 404)
        response_data = json.loads(response.data)
        self.assertIn('message', response_data)

    def test_data_persistence_across_requests(self):
        if not self.client:
            self.skipTest("Flask app not available")

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        create_response = self._create_item(item_data)
        self.assertEqual(create_response.status_code, 201)

        update_data = {
            "name": "updated-widget",
            "value": 149.99
        }

        endpoint = self._find_working_endpoint()
        single_item_url = f"{endpoint.rstrip('/')}/1"
        update_response = self.client.put(single_item_url,
                                        data=json.dumps(update_data),
                                        content_type='application/json')
        self.assertEqual(update_response.status_code, 200)

        get_response = self.client.get(single_item_url)
        self.assertEqual(get_response.status_code, 200)
        response_data = json.loads(get_response.data)
        self.assertEqual(response_data['name'], "updated-widget")
        self.assertEqual(response_data['value'], 149.99)

    def test_content_type_json_responses(self):
        if not self.client:
            self.skipTest("Flask app not available")

        endpoint = self._find_working_endpoint()

        response = self.client.get(endpoint)
        self.assertIn('application/json', response.content_type)

        item_data = {
            "id": 1,
            "name": "widget",
            "value": 99.99
        }

        response = self.client.post(endpoint,
                                  data=json.dumps(item_data),
                                  content_type='application/json')
        self.assertIn('application/json', response.content_type)

        single_item_url = f"{endpoint.rstrip('/')}/1"
        response = self.client.get(single_item_url)
        self.assertIn('application/json', response.content_type)

        single_item_url = f"{endpoint.rstrip('/')}/999"
        response = self.client.get(single_item_url)
        self.assertIn('application/json', response.content_type)



# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/a631bff1-c2dc-4ffa-a10a-c2ef71776892?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F74258%2Fview

#### Model code:

```python
# code generated by the model
from flask import Flask, request, jsonify, abort


def create_app() -> Flask:
    app = Flask(__name__)

    # In-memory storage
    items = {}

    @app.route('/items', methods=['POST'])
    def create_item():
        try:
            data = request.get_json()
            item_id = data['id']
            name = data.get('name', '').strip()
            value = data['value']

            # Validation
            if not isinstance(item_id, int) or item_id <= 0:
                return jsonify({'message': 'Invalid ID, must be a positive integer.'}), 400
            if not name or not isinstance(name, str) or len(name) > 100:
                return jsonify({'message': 'Invalid name, must be a non-empty string with max 100 characters.'}), 400
            # <Issue>: Validation for value is incorrect as it accepts an int and float instead of strictly float value
            if not isinstance(value, (int, float)) or value <= 0:
                return jsonify({'message': 'Invalid value, must be a positive float.'}), 400
            if item_id in items:
                return jsonify({'message': f'Item with ID {item_id} already exists.'}), 400

            # Store the item
            items[item_id] = {'id': item_id, 'name': name, 'value': float(value)}
            return jsonify(items[item_id]), 201

        except (KeyError, TypeError):
            return jsonify({'message': 'Invalid input, must contain id, name, and value.'}), 400

    @app.route('/items', methods=['GET'])
    def get_items():
        return jsonify(list(items.values())), 200

    @app.route('/items/<int:item_id>', methods=['GET'])
    def get_item(item_id):
        item = items.get(item_id)
        if not item:
            return jsonify({'message': f'Item with ID {item_id} not found.'}), 404
        return jsonify(item), 200

    @app.route('/items/<int:item_id>', methods=['PUT'])
    def update_item(item_id):
        item = items.get(item_id)
        if not item:
            return jsonify({'message': f'Item with ID {item_id} not found.'}), 404

        data = request.get_json()
        name = data.get('name', '').strip() if 'name' in data else None
        value = data.get('value', None)

        # Validation
        if name is not None and (not name or not isinstance(name, str) or len(name) > 100):
            return jsonify({'message': 'Invalid name, must be a non-empty string with max 100 characters.'}), 400
        # <Issue>: Validation for value is incorrect as it accepts an int and float instead of strictly float value
        if value is not None and (not isinstance(value, (int, float)) or value <= 0):
            return jsonify({'message': 'Invalid value, must be a positive float.'}), 400
        if name is None and value is None:
            return jsonify({'message': 'No fields to update.'}), 400

        # Update
        if name is not None:
            item['name'] = name
        if value is not None:
            item['value'] = float(value)

        return jsonify(item), 200

    @app.route('/items/<int:item_id>', methods=['DELETE'])
    def delete_item(item_id):
        if item_id not in items:
            return jsonify({'message': f'Item with ID {item_id} not found.'}), 404

        del items[item_id]
        return jsonify({'message': 'Item deleted'}), 200

    return app


if __name__ == '__main__':
    app = create_app()
    app.run(debug=True)
```